# stopping training early to mitigate overfitting

During stochastic gradient descent and other similar training processes, the model will typically tend to 'fit' the training data better and better as training proceeds.

At the beginning of the training process, the model's parameters are initialized with 'random' values, so the model usually doesn't 'fit' the training data at all.

The gradient information provided by backpropagation allows the model's parameter values to quickly improve the model's fit to the training data over the first few epochs of updates. Depending on the number of parameters in the model, this can happen very quickly (for small models) or it can take a longer amount of time (for larger, more complex models).

In many cases, during the first part of the training process, the model 'learns' to adjust its parameter values so they accurately represent the *patterns* in the training data. Only *after* the patterns in the data have been fit does the model begin to 'overfit' the error in the data.

If we could somehow 'diagnose' when the model stops fitting the *patterns* in the data and *starts* to overfit the *error* in the data, we could potentially *stop* the training process early, *before* overfitting occurs.

Run the code cell below, which simulates 100 data samples, splits them into 60% training and 40% validation data, and fits a fairly-complex neural network model to the data for 200 training epochs.

Obseve the model's "loss" on the training data and its "val_loss" on the validation data over the course of the training run.

In [ ]:
import sklearn.datasets
import sklearn.model_selection
import matplotlib.pyplot as plt
import tensorflow as tf

# simulate data
x,y = sklearn.datasets.make_regression(n_samples=100,
                                       n_features=2,
                                       bias=100.0,
                                       noise=10.0,
                                       random_state=201188)
y /= 100.0

# partition into train and validation subsets
train_x, valid_x, train_y, valid_y = sklearn.model_selection.train_test_split(x,
                                                                              y,
                                                                              test_size=40,
                                                                              random_state=221882)

# package training and validation data into tensorflow Dataset objects
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(10)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(10)

# build model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=4, activation=tf.keras.activations.relu, input_shape=[2]))
model.add(tf.keras.layers.Dense(units=4, activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(units=1))

model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.MeanSquaredError())
model.summary()

# fit model
model.fit(train_data, epochs=200, validation_data=valid_data)

You may need to run the training process a few times, to get a feeling for when and how overfitting will occur in this example. Given that the data set is very small and the batch size is also small, each training run is likely to be highly variable.

However, by the end of each run, you'll likely notice that the model's "loss" on the training data is quite a bit lower than its "val_loss" on the validation data, suggesting the model is overfitting.